In [1]:
import proplot as pplt
import fitsio
import numpy as np
import des_y6utils.mdet
import ngmix
import scipy.interpolate

In [2]:
def _process_tile(fname, sn, col, cutver):
    d = fitsio.read(fname)
    cols = ["w", "we1", "we2", "w2", "we12", "we22", "w2e12", "w2e22"]
    dtype = []
    for tail in ["noshear", "1p", "1m", "2p", "2m"]:
        dtype += [
            (col + "_" + tail, "f8")
            for col in cols
        ]
    # print(dtype)
    res = np.zeros(1, dtype=dtype)
    
    d = d[des_y6utils.mdet.make_mdet_cuts(d, cutver)]
    w = 1.0/(sn**2 + 0.5*d[col + "_g_cov_1_1"]**2 + 0.5*d[col + "_g_cov_2_2"]**2)
    for tail in ["noshear", "1p", "1m", "2p", "2m"]:
        msk = (d["mdet_step"] == tail)
        res["w_" + tail] = np.sum(w[msk])
        res["w2_" + tail] = np.sum(w[msk]**2)
        res["we1_" + tail] = np.sum(w[msk] * d[col + "_g_1"][msk])
        res["we2_" + tail] = np.sum(w[msk] * d[col + "_g_2"][msk])
        res["we12_" + tail] = np.sum(w[msk] * d[col + "_g_1"][msk]**2)
        res["we22_" + tail] = np.sum(w[msk] * d[col + "_g_2"][msk]**2)
        res["w2e12_" + tail] = np.sum(w[msk]**2 * d[col + "_g_1"][msk]**2)
        res["w2e22_" + tail] = np.sum(w[msk]**2 * d[col + "_g_2"][msk]**2)
    return res

In [3]:
import glob
import esutil
import joblib

fnames_gauss_grizdet = glob.glob("blinded_data_test_v1/*.fits")

In [4]:
import os

fnames_wmom = []
for fname in esutil.pbar.PBar(fnames_gauss_grizdet):
    _fnames = glob.glob(os.path.join(
        "..",
        "2022_08_24_blind_and_test_final_cats",
        "blinded_data",
        fname.split("/")[1][:len("DES2328+0043")] + "*.fits"
    ))
    assert len(_fnames) == 1
    fnames_wmom.append(_fnames[0])

fnames_gauss_rizdet = []
for fname in esutil.pbar.PBar(fnames_gauss_grizdet):
    _fnames = glob.glob(os.path.join(
        "blinded_data_test_v2",
        fname.split("/")[1][:len("DES2328+0043")] + "*.fits"
    ))
    assert len(_fnames) == 1
    fnames_gauss_rizdet.append(_fnames[0])

|####################| 1000/1000 100% [elapsed: 00:14 left: 00:00]
|####################| 1000/1000 100% [elapsed: 00:01 left: 00:00]


In [22]:
def _meas_jobs(d):
    with joblib.Parallel(n_jobs=12, backend="loky", verbose=100) as exc:
        d = exc(d)
    d = np.hstack(d)

    R11 = (
        np.sum(d["we1_1p"]) / np.sum(d["w_1p"])
        - np.sum(d["we1_1m"]) / np.sum(d["w_1m"])
    )/0.02

    R22 = (
        np.sum(d["we2_2p"]) / np.sum(d["w_2p"])
        - np.sum(d["we2_2m"]) / np.sum(d["w_2m"])
    )/0.02

    mn_e1 = np.sum(d["we1_noshear"]) / np.sum(d["w_noshear"])/R11
    var_e1 = np.sum(d["we12_noshear"]) / np.sum(d["w_noshear"]) / R11**2
    print(mn_e1, R11, R22, np.sqrt(var_e1 - mn_e1**2))
    
    return d

In [23]:
num_files = 1000

In [24]:
d = [
    joblib.delayed(_process_tile)(fname, 0.07, "wmom", "2")
    for fname in fnames_wmom
][:num_files]
dwmom = _meas_jobs(d)

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:   16.3s
[Parallel(n_jobs=12)]: Done   2 tasks      | elapsed:   17.6s
[Parallel(n_jobs=12)]: Done   3 tasks      | elapsed:   18.7s
[Parallel(n_jobs=12)]: Done   4 tasks      | elapsed:  1.0min
[Parallel(n_jobs=12)]: Done   5 tasks      | elapsed:  1.0min
[Parallel(n_jobs=12)]: Done   6 tasks      | elapsed:  1.0min
[Parallel(n_jobs=12)]: Done   7 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done   9 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done  10 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done  11 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done  12 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done  13 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done  14 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done  15 tasks      | elapsed:  

[Parallel(n_jobs=12)]: Done 132 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 133 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 134 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 135 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 136 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 137 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 139 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 140 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 141 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 142 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 143 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 144 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 145 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 146 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 147 tasks      | elapsed:  3.0min
[Paralle

[Parallel(n_jobs=12)]: Done 265 tasks      | elapsed:  4.1min
[Parallel(n_jobs=12)]: Done 266 tasks      | elapsed:  4.1min
[Parallel(n_jobs=12)]: Done 267 tasks      | elapsed:  4.1min
[Parallel(n_jobs=12)]: Done 268 tasks      | elapsed:  4.1min
[Parallel(n_jobs=12)]: Done 269 tasks      | elapsed:  4.2min
[Parallel(n_jobs=12)]: Done 270 tasks      | elapsed:  4.2min
[Parallel(n_jobs=12)]: Done 271 tasks      | elapsed:  4.2min
[Parallel(n_jobs=12)]: Done 272 tasks      | elapsed:  4.2min
[Parallel(n_jobs=12)]: Done 273 tasks      | elapsed:  4.2min
[Parallel(n_jobs=12)]: Done 274 tasks      | elapsed:  4.2min
[Parallel(n_jobs=12)]: Done 275 tasks      | elapsed:  4.2min
[Parallel(n_jobs=12)]: Done 276 tasks      | elapsed:  4.2min
[Parallel(n_jobs=12)]: Done 277 tasks      | elapsed:  4.2min
[Parallel(n_jobs=12)]: Done 278 tasks      | elapsed:  4.3min
[Parallel(n_jobs=12)]: Done 279 tasks      | elapsed:  4.3min
[Parallel(n_jobs=12)]: Done 280 tasks      | elapsed:  4.3min
[Paralle

[Parallel(n_jobs=12)]: Done 398 tasks      | elapsed:  5.4min
[Parallel(n_jobs=12)]: Done 399 tasks      | elapsed:  5.4min
[Parallel(n_jobs=12)]: Done 400 tasks      | elapsed:  5.4min
[Parallel(n_jobs=12)]: Done 401 tasks      | elapsed:  5.4min
[Parallel(n_jobs=12)]: Done 402 tasks      | elapsed:  5.4min
[Parallel(n_jobs=12)]: Done 403 tasks      | elapsed:  5.5min
[Parallel(n_jobs=12)]: Done 404 tasks      | elapsed:  5.5min
[Parallel(n_jobs=12)]: Done 405 tasks      | elapsed:  5.5min
[Parallel(n_jobs=12)]: Done 406 tasks      | elapsed:  5.5min
[Parallel(n_jobs=12)]: Done 407 tasks      | elapsed:  5.5min
[Parallel(n_jobs=12)]: Done 408 tasks      | elapsed:  5.5min
[Parallel(n_jobs=12)]: Done 409 tasks      | elapsed:  5.5min
[Parallel(n_jobs=12)]: Done 410 tasks      | elapsed:  5.5min
[Parallel(n_jobs=12)]: Done 411 tasks      | elapsed:  5.5min
[Parallel(n_jobs=12)]: Done 412 tasks      | elapsed:  5.6min
[Parallel(n_jobs=12)]: Done 413 tasks      | elapsed:  5.6min
[Paralle

[Parallel(n_jobs=12)]: Done 531 tasks      | elapsed:  6.7min
[Parallel(n_jobs=12)]: Done 532 tasks      | elapsed:  6.7min
[Parallel(n_jobs=12)]: Done 533 tasks      | elapsed:  6.7min
[Parallel(n_jobs=12)]: Done 534 tasks      | elapsed:  6.7min
[Parallel(n_jobs=12)]: Done 535 tasks      | elapsed:  6.7min
[Parallel(n_jobs=12)]: Done 536 tasks      | elapsed:  6.7min
[Parallel(n_jobs=12)]: Done 537 tasks      | elapsed:  6.7min
[Parallel(n_jobs=12)]: Done 538 tasks      | elapsed:  6.7min
[Parallel(n_jobs=12)]: Done 539 tasks      | elapsed:  6.7min
[Parallel(n_jobs=12)]: Done 540 tasks      | elapsed:  6.7min
[Parallel(n_jobs=12)]: Done 541 tasks      | elapsed:  6.8min
[Parallel(n_jobs=12)]: Done 542 tasks      | elapsed:  6.8min
[Parallel(n_jobs=12)]: Done 543 tasks      | elapsed:  6.8min
[Parallel(n_jobs=12)]: Done 544 tasks      | elapsed:  6.8min
[Parallel(n_jobs=12)]: Done 545 tasks      | elapsed:  6.8min
[Parallel(n_jobs=12)]: Done 546 tasks      | elapsed:  6.8min
[Paralle

[Parallel(n_jobs=12)]: Done 664 tasks      | elapsed:  7.9min
[Parallel(n_jobs=12)]: Done 665 tasks      | elapsed:  7.9min
[Parallel(n_jobs=12)]: Done 666 tasks      | elapsed:  7.9min
[Parallel(n_jobs=12)]: Done 667 tasks      | elapsed:  7.9min
[Parallel(n_jobs=12)]: Done 668 tasks      | elapsed:  7.9min
[Parallel(n_jobs=12)]: Done 669 tasks      | elapsed:  7.9min
[Parallel(n_jobs=12)]: Done 670 tasks      | elapsed:  8.0min
[Parallel(n_jobs=12)]: Done 671 tasks      | elapsed:  8.0min
[Parallel(n_jobs=12)]: Done 672 tasks      | elapsed:  8.0min
[Parallel(n_jobs=12)]: Done 673 tasks      | elapsed:  8.0min
[Parallel(n_jobs=12)]: Done 674 tasks      | elapsed:  8.0min
[Parallel(n_jobs=12)]: Done 675 tasks      | elapsed:  8.0min
[Parallel(n_jobs=12)]: Done 676 tasks      | elapsed:  8.0min
[Parallel(n_jobs=12)]: Done 677 tasks      | elapsed:  8.0min
[Parallel(n_jobs=12)]: Done 678 tasks      | elapsed:  8.0min
[Parallel(n_jobs=12)]: Done 679 tasks      | elapsed:  8.0min
[Paralle

[Parallel(n_jobs=12)]: Done 797 tasks      | elapsed:  9.1min
[Parallel(n_jobs=12)]: Done 798 tasks      | elapsed:  9.1min
[Parallel(n_jobs=12)]: Done 799 tasks      | elapsed:  9.1min
[Parallel(n_jobs=12)]: Done 800 tasks      | elapsed:  9.2min
[Parallel(n_jobs=12)]: Done 801 tasks      | elapsed:  9.2min
[Parallel(n_jobs=12)]: Done 802 tasks      | elapsed:  9.2min
[Parallel(n_jobs=12)]: Done 803 tasks      | elapsed:  9.2min
[Parallel(n_jobs=12)]: Done 804 tasks      | elapsed:  9.2min
[Parallel(n_jobs=12)]: Done 805 tasks      | elapsed:  9.2min
[Parallel(n_jobs=12)]: Done 806 tasks      | elapsed:  9.2min
[Parallel(n_jobs=12)]: Done 807 tasks      | elapsed:  9.2min
[Parallel(n_jobs=12)]: Done 808 tasks      | elapsed:  9.2min
[Parallel(n_jobs=12)]: Done 809 tasks      | elapsed:  9.2min
[Parallel(n_jobs=12)]: Done 810 tasks      | elapsed:  9.2min
[Parallel(n_jobs=12)]: Done 811 tasks      | elapsed:  9.2min
[Parallel(n_jobs=12)]: Done 812 tasks      | elapsed:  9.3min
[Paralle

[Parallel(n_jobs=12)]: Done 930 tasks      | elapsed: 10.3min
[Parallel(n_jobs=12)]: Done 931 tasks      | elapsed: 10.3min
[Parallel(n_jobs=12)]: Done 932 tasks      | elapsed: 10.4min
[Parallel(n_jobs=12)]: Done 933 tasks      | elapsed: 10.4min
[Parallel(n_jobs=12)]: Done 934 tasks      | elapsed: 10.4min
[Parallel(n_jobs=12)]: Done 935 tasks      | elapsed: 10.4min
[Parallel(n_jobs=12)]: Done 936 tasks      | elapsed: 10.4min
[Parallel(n_jobs=12)]: Done 937 tasks      | elapsed: 10.4min
[Parallel(n_jobs=12)]: Done 938 tasks      | elapsed: 10.4min
[Parallel(n_jobs=12)]: Done 939 tasks      | elapsed: 10.4min
[Parallel(n_jobs=12)]: Done 940 tasks      | elapsed: 10.5min
[Parallel(n_jobs=12)]: Done 941 tasks      | elapsed: 10.5min
[Parallel(n_jobs=12)]: Done 942 tasks      | elapsed: 10.5min
[Parallel(n_jobs=12)]: Done 943 tasks      | elapsed: 10.5min
[Parallel(n_jobs=12)]: Done 944 tasks      | elapsed: 10.5min
[Parallel(n_jobs=12)]: Done 945 tasks      | elapsed: 10.5min
[Paralle

In [25]:
d = [
    joblib.delayed(_process_tile)(fname, 0.23, "gauss", "3")
    for fname in fnames_gauss_grizdet
][:num_files]
dgauss_griz = _meas_jobs(d)

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.9s
[Parallel(n_jobs=12)]: Done   2 tasks      | elapsed:    2.3s
[Parallel(n_jobs=12)]: Done   3 tasks      | elapsed:    4.0s
[Parallel(n_jobs=12)]: Done   4 tasks      | elapsed:    4.2s
[Parallel(n_jobs=12)]: Done   5 tasks      | elapsed:    4.3s
[Parallel(n_jobs=12)]: Done   6 tasks      | elapsed:    4.4s
[Parallel(n_jobs=12)]: Done   7 tasks      | elapsed:    4.5s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    4.5s
[Parallel(n_jobs=12)]: Done   9 tasks      | elapsed:    4.6s
[Parallel(n_jobs=12)]: Done  10 tasks      | elapsed:    4.6s
[Parallel(n_jobs=12)]: Done  11 tasks      | elapsed:    4.7s
[Parallel(n_jobs=12)]: Done  12 tasks      | elapsed:    4.8s
[Parallel(n_jobs=12)]: Done  13 tasks      | elapsed:    5.0s
[Parallel(n_jobs=12)]: Done  14 tasks      | elapsed:    5.3s
[Parallel(n_jobs=12)]: Done  15 tasks      | elapsed:  

[Parallel(n_jobs=12)]: Done 133 tasks      | elapsed:   34.3s
[Parallel(n_jobs=12)]: Done 134 tasks      | elapsed:   34.6s
[Parallel(n_jobs=12)]: Done 135 tasks      | elapsed:   34.7s
[Parallel(n_jobs=12)]: Done 136 tasks      | elapsed:   34.9s
[Parallel(n_jobs=12)]: Done 137 tasks      | elapsed:   35.1s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:   35.4s
[Parallel(n_jobs=12)]: Done 139 tasks      | elapsed:   35.9s
[Parallel(n_jobs=12)]: Done 140 tasks      | elapsed:   36.2s
[Parallel(n_jobs=12)]: Done 141 tasks      | elapsed:   36.7s
[Parallel(n_jobs=12)]: Done 142 tasks      | elapsed:   36.8s
[Parallel(n_jobs=12)]: Done 143 tasks      | elapsed:   36.9s
[Parallel(n_jobs=12)]: Done 144 tasks      | elapsed:   37.0s
[Parallel(n_jobs=12)]: Done 145 tasks      | elapsed:   37.2s
[Parallel(n_jobs=12)]: Done 146 tasks      | elapsed:   37.5s
[Parallel(n_jobs=12)]: Done 147 tasks      | elapsed:   37.7s
[Parallel(n_jobs=12)]: Done 148 tasks      | elapsed:   37.7s
[Paralle

[Parallel(n_jobs=12)]: Done 267 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 268 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 269 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 270 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 271 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 272 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 273 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 274 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 275 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 276 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 277 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 278 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 279 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 280 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 281 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 282 tasks      | elapsed:  1.2min
[Paralle

[Parallel(n_jobs=12)]: Done 401 tasks      | elapsed:  1.6min
[Parallel(n_jobs=12)]: Done 402 tasks      | elapsed:  1.6min
[Parallel(n_jobs=12)]: Done 403 tasks      | elapsed:  1.6min
[Parallel(n_jobs=12)]: Done 404 tasks      | elapsed:  1.6min
[Parallel(n_jobs=12)]: Done 405 tasks      | elapsed:  1.6min
[Parallel(n_jobs=12)]: Done 406 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 407 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 408 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 409 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 410 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 411 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 412 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 413 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 414 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 415 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 416 tasks      | elapsed:  1.7min
[Paralle

[Parallel(n_jobs=12)]: Done 534 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 535 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 536 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 537 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 538 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 539 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 540 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 541 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 542 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 543 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 544 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 545 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 546 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 547 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 548 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 549 tasks      | elapsed:  2.2min
[Paralle

[Parallel(n_jobs=12)]: Done 667 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 668 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 669 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 670 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 671 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 672 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 673 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 674 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 675 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 676 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 677 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 678 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 679 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 680 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 681 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 682 tasks      | elapsed:  2.8min
[Paralle

[Parallel(n_jobs=12)]: Done 800 tasks      | elapsed:  3.2min
[Parallel(n_jobs=12)]: Done 801 tasks      | elapsed:  3.2min
[Parallel(n_jobs=12)]: Done 802 tasks      | elapsed:  3.2min
[Parallel(n_jobs=12)]: Done 803 tasks      | elapsed:  3.2min
[Parallel(n_jobs=12)]: Done 804 tasks      | elapsed:  3.2min
[Parallel(n_jobs=12)]: Done 805 tasks      | elapsed:  3.2min
[Parallel(n_jobs=12)]: Done 806 tasks      | elapsed:  3.2min
[Parallel(n_jobs=12)]: Done 807 tasks      | elapsed:  3.3min
[Parallel(n_jobs=12)]: Done 808 tasks      | elapsed:  3.3min
[Parallel(n_jobs=12)]: Done 809 tasks      | elapsed:  3.3min
[Parallel(n_jobs=12)]: Done 810 tasks      | elapsed:  3.3min
[Parallel(n_jobs=12)]: Done 811 tasks      | elapsed:  3.3min
[Parallel(n_jobs=12)]: Done 812 tasks      | elapsed:  3.3min
[Parallel(n_jobs=12)]: Done 813 tasks      | elapsed:  3.3min
[Parallel(n_jobs=12)]: Done 814 tasks      | elapsed:  3.3min
[Parallel(n_jobs=12)]: Done 815 tasks      | elapsed:  3.3min
[Paralle

[Parallel(n_jobs=12)]: Done 935 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 936 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 937 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 938 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 939 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 940 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 941 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 942 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 943 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 944 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 945 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 946 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 947 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 948 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 949 tasks      | elapsed:  3.8min
[Parallel(n_jobs=12)]: Done 950 tasks      | elapsed:  3.8min
[Paralle

In [26]:
d = [
    joblib.delayed(_process_tile)(fname, 0.23, "gauss", "3")
    for fname in fnames_gauss_rizdet
][:num_files]
dgauss_riz = _meas_jobs(d)

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.9s
[Parallel(n_jobs=12)]: Done   2 tasks      | elapsed:    2.5s
[Parallel(n_jobs=12)]: Done   3 tasks      | elapsed:    3.5s
[Parallel(n_jobs=12)]: Done   4 tasks      | elapsed:    3.5s
[Parallel(n_jobs=12)]: Done   5 tasks      | elapsed:    4.5s
[Parallel(n_jobs=12)]: Done   6 tasks      | elapsed:    4.6s
[Parallel(n_jobs=12)]: Done   7 tasks      | elapsed:    4.7s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    4.7s
[Parallel(n_jobs=12)]: Done   9 tasks      | elapsed:    4.8s
[Parallel(n_jobs=12)]: Done  10 tasks      | elapsed:    4.9s
[Parallel(n_jobs=12)]: Done  11 tasks      | elapsed:    5.1s
[Parallel(n_jobs=12)]: Done  12 tasks      | elapsed:    5.1s
[Parallel(n_jobs=12)]: Done  13 tasks      | elapsed:    5.3s
[Parallel(n_jobs=12)]: Done  14 tasks      | elapsed:    5.5s
[Parallel(n_jobs=12)]: Done  15 tasks      | elapsed:  

[Parallel(n_jobs=12)]: Done 133 tasks      | elapsed:   37.0s
[Parallel(n_jobs=12)]: Done 134 tasks      | elapsed:   37.1s
[Parallel(n_jobs=12)]: Done 135 tasks      | elapsed:   37.2s
[Parallel(n_jobs=12)]: Done 136 tasks      | elapsed:   37.2s
[Parallel(n_jobs=12)]: Done 137 tasks      | elapsed:   37.6s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:   37.6s
[Parallel(n_jobs=12)]: Done 139 tasks      | elapsed:   38.9s
[Parallel(n_jobs=12)]: Done 140 tasks      | elapsed:   39.1s
[Parallel(n_jobs=12)]: Done 141 tasks      | elapsed:   39.1s
[Parallel(n_jobs=12)]: Done 142 tasks      | elapsed:   39.2s
[Parallel(n_jobs=12)]: Done 143 tasks      | elapsed:   39.7s
[Parallel(n_jobs=12)]: Done 144 tasks      | elapsed:   40.2s
[Parallel(n_jobs=12)]: Done 145 tasks      | elapsed:   40.3s
[Parallel(n_jobs=12)]: Done 146 tasks      | elapsed:   40.3s
[Parallel(n_jobs=12)]: Done 147 tasks      | elapsed:   40.3s
[Parallel(n_jobs=12)]: Done 148 tasks      | elapsed:   40.4s
[Paralle

[Parallel(n_jobs=12)]: Done 266 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 267 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 268 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 269 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 270 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 271 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 272 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 273 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 274 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 275 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 276 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 277 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 278 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 279 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 280 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 281 tasks      | elapsed:  1.2min
[Paralle

[Parallel(n_jobs=12)]: Done 402 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 403 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 404 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 405 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 406 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 407 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 408 tasks      | elapsed:  1.8min
[Parallel(n_jobs=12)]: Done 409 tasks      | elapsed:  1.8min
[Parallel(n_jobs=12)]: Done 410 tasks      | elapsed:  1.8min
[Parallel(n_jobs=12)]: Done 411 tasks      | elapsed:  1.8min
[Parallel(n_jobs=12)]: Done 412 tasks      | elapsed:  1.8min
[Parallel(n_jobs=12)]: Done 413 tasks      | elapsed:  1.8min
[Parallel(n_jobs=12)]: Done 414 tasks      | elapsed:  1.8min
[Parallel(n_jobs=12)]: Done 415 tasks      | elapsed:  1.8min
[Parallel(n_jobs=12)]: Done 416 tasks      | elapsed:  1.8min
[Parallel(n_jobs=12)]: Done 417 tasks      | elapsed:  1.8min
[Paralle

[Parallel(n_jobs=12)]: Done 538 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 539 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 540 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 541 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 542 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 543 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 544 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 545 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 546 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 547 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 548 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 549 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 550 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 551 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 552 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 553 tasks      | elapsed:  2.3min
[Paralle

[Parallel(n_jobs=12)]: Done 671 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 672 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 673 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 674 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 675 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 676 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 677 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 678 tasks      | elapsed:  2.8min
[Parallel(n_jobs=12)]: Done 679 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 680 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 681 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 682 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 683 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 684 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 685 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 686 tasks      | elapsed:  2.9min
[Paralle

[Parallel(n_jobs=12)]: Done 806 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 807 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 808 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 809 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 810 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 811 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 812 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 813 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 814 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 815 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 816 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 817 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 818 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 819 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 820 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 821 tasks      | elapsed:  3.4min
[Paralle

[Parallel(n_jobs=12)]: Done 939 tasks      | elapsed:  3.9min
[Parallel(n_jobs=12)]: Done 940 tasks      | elapsed:  3.9min
[Parallel(n_jobs=12)]: Done 941 tasks      | elapsed:  3.9min
[Parallel(n_jobs=12)]: Done 942 tasks      | elapsed:  3.9min
[Parallel(n_jobs=12)]: Done 943 tasks      | elapsed:  3.9min
[Parallel(n_jobs=12)]: Done 944 tasks      | elapsed:  3.9min
[Parallel(n_jobs=12)]: Done 945 tasks      | elapsed:  4.0min
[Parallel(n_jobs=12)]: Done 946 tasks      | elapsed:  4.0min
[Parallel(n_jobs=12)]: Done 947 tasks      | elapsed:  4.0min
[Parallel(n_jobs=12)]: Done 948 tasks      | elapsed:  4.0min
[Parallel(n_jobs=12)]: Done 949 tasks      | elapsed:  4.0min
[Parallel(n_jobs=12)]: Done 950 tasks      | elapsed:  4.0min
[Parallel(n_jobs=12)]: Done 951 tasks      | elapsed:  4.0min
[Parallel(n_jobs=12)]: Done 952 tasks      | elapsed:  4.0min
[Parallel(n_jobs=12)]: Done 953 tasks      | elapsed:  4.0min
[Parallel(n_jobs=12)]: Done 954 tasks      | elapsed:  4.0min
[Paralle

In [27]:
def shear_prec(d, quiet=True):
    R11 = (
        np.sum(d["we1_1p"]) / np.sum(d["w_1p"])
        - np.sum(d["we1_1m"]) / np.sum(d["w_1m"])
    )/0.02
    mn_e1 = np.sum(d["we1_noshear"]) / np.sum(d["w_noshear"])
    sigma_e1 = np.sqrt(
        (np.sum(d["w2e12_noshear"]) - np.sum(d["w2_noshear"])*mn_e1**2) 
        / (np.sum(d["w2_noshear"]) * R11**2)
    )
    
    R22 = (
        np.sum(d["we2_2p"]) / np.sum(d["w_2p"])
        - np.sum(d["we2_2m"]) / np.sum(d["w_2m"])
    )/0.02
    mn_e2 = np.sum(d["we2_noshear"]) / np.sum(d["w_noshear"])
    sigma_e2 = np.sqrt(
        (np.sum(d["w2e22_noshear"]) - np.sum(d["w2_noshear"])*mn_e2**2) 
        / (np.sum(d["w2_noshear"]) * R22**2)
    )
    neff = np.sum(d["w_noshear"])**2 / np.sum(d["w2_noshear"])
    if not quiet:
        print(
            "\tsn_e1|sn_e2|neff|prec: %0.4f|%0.4f|%0.4g|%0.4g" % (
                sigma_e1, sigma_e2, neff, sigma_e1/np.sqrt(neff)/1e-4
            )
        )
    return np.array([sigma_e1/np.sqrt(neff)/1e-4, np.sqrt((sigma_e1**2 + sigma_e2**2)/2), neff])
    # return sigma_e1/np.sqrt(neff)/1e-4 #, sigma_e1, sigma_e2, neff])


def bootstrap(d, func, n_boot=500, seed=None):
    rng = seed or np.random.default_rng(seed)
    
    res = []
    for _ in range(n_boot):
        _d = d[rng.choice(len(d), size=len(d), replace=True)]
        res.append(np.atleast_1d(func(_d)))
    res = np.vstack(res)
    return np.atleast_1d(func(d)), np.std(res, axis=0)

swmom = bootstrap(dwmom, shear_prec)
sgriz = bootstrap(dgauss_griz, shear_prec)
sriz = bootstrap(dgauss_riz, shear_prec)

def _print_sum(s, name):
    print(name + ":")
    print("\tshape noise: %0.3f +/- %0.3f" % (
            s[0][1], s[1][1]
    ))
    print("\tneff: %0.2f +/- %0.2f" % (
            s[0][2]/1e6/num_files*1e4, s[1][2]/1e6/num_files*1e4
    ))
    print(
        "\tprec: %0.3f +/- %0.3f" % (
            s[0][0], s[1][0]
    ))
    

_print_sum(swmom, "wmom")
_print_sum(sgriz, "gauss_griz")
_print_sum(sriz, "gauss_riz")


def bootstrap_pair(d1, d2, func, n_boot=500, seed=None):
    rng = seed or np.random.default_rng(seed)
    
    res = []
    for _ in range(n_boot):
        inds = rng.choice(len(d1), size=len(d1), replace=True)
        _d1 = d1[inds]
        _d2 = d2[inds]
        res.append(np.atleast_1d(func(_d1)) / np.atleast_1d(func(_d2)))
    res = np.vstack(res)
    return np.atleast_1d(func(d1)) / np.atleast_1d(func(d2)), np.std(res, axis=0)

bootstrap_pair(dgauss_riz, dgauss_griz, shear_prec)

# print("wmom:")
# shear_prec(dwmom, quiet=False);
# print("gauss det griz:")
# shear_prec(dgauss_griz, quiet=False);
# print("gauss det riz:")
# shear_prec(dgauss_riz, quiet=False);
# print(
#     "gauss_griz/wmom: %0.3f\ngauss_riz/wmom: %0.3f" % ( 
#     shear_prec(dgauss_griz)/shear_prec(dwmom), 
#     shear_prec(dgauss_riz)/shear_prec(dwmom)
# ))

wmom:
	shape noise: 0.352 +/- 0.001
	neff: 149.34 +/- 0.98
	prec: 0.911 +/- 0.004
gauss_griz:
	shape noise: 0.302 +/- 0.000
	neff: 148.00 +/- 0.94
	prec: 0.788 +/- 0.003
gauss_riz:
	shape noise: 0.300 +/- 0.000
	neff: 147.50 +/- 0.91
	prec: 0.783 +/- 0.003


(array([0.99368134, 0.9937623 , 0.99660615]),
 array([1.38692196e-03, 9.48253005e-04, 7.63589100e-05]))

In [ ]:
def sn_h12(e, w=None):
    if w is None:
        w = np.ones_like(e)
        
    return np.sqrt(np.sum((w*e)**2)/np.sum(w**2))

def neff_h12(e, w=None):
    if w is None:
        w = np.ones_like(e)
    return np.sum(w)**2/np.sum(w**2)

def prec_h12(e, R, w=None):
    if w is None:
        w = np.ones_like(e)

    return sn_h12(e, w=w)/np.sqrt(neff_h12(e, w=w))/R

In [ ]:
def _comp_stats(d, msk, w):
    R = _compute_r(d, msk=msk, w=w)
    _msk = msk & (d["mdet_step"] == "noshear")
    prec_nrm = 0.07/R/np.sqrt(0.3 * d.shape[0] / 5)
    return (
        np.sum(_msk),
        sn_h12(d["wmom_g_1"][_msk], w=w[_msk])/R,
        neff_h12(d["wmom_g_1"][_msk], w=w[_msk])/(d.shape[0]/5),
        prec_h12(d["wmom_g_1"][_msk], R, w=w[_msk])/prec_nrm,
    )   